In [ ]:
#%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Inlezen

De eerste data set bevat de aanwezigheidsgegevens uit Canvas van studenten bij de lessen Programming gedurende een blok. Het `Student ID` hier is het identificatienummer van __Canvas__.

In [ ]:
df_prog = pd.read_csv('pandas_attendance_prog.csv', delimiter=';')

In [ ]:
df_prog.head()

In [ ]:
df_prog.drop(['Course ID', 'Teacher ID'], axis=1, inplace=True)

In [ ]:
df_prog.columns

In [ ]:
df_prog.rename(columns={'Teacher Name': 'Docent'}, inplace=True)

In [ ]:
df_prog.head()

De tweede dataset bevat gegevens over de gebruikers van Canvas, waaronder een studentnummer in het geval van een student. (Veel gegevens, zoals achternaam en emailadres, zijn hier verwijderd vanwege privacy).

In [ ]:
df_users = pd.read_table('pandas_attendance_users.csv', sep=';', encoding='utf-8')

In [ ]:
df_users.head()

## Lege waarden

In [ ]:
df_users.isnull().head()

In [ ]:
len(df_users)

In [ ]:
len(df_users.dropna())

In [ ]:
len(df_users.dropna(subset=['status']))

In [ ]:
df_users.dropna(inplace=True)

In [ ]:
df_users.head()

In [ ]:
df_users['user_id'] = df_users['user_id'].astype(int)

In [ ]:
df_users.set_index('user_id', inplace=True)

In [ ]:
df_users.head()

## Multilevel indexing

De laatste dataset bevat de inschrijvingen van een student in bepaalde vakken en de klas waarin hij of zij zit uit Osiris.

In [ ]:
df_students = pd.read_csv('pandas_attendance_students.csv', sep=';')

In [ ]:
df_students.head()

In [ ]:
len(df_students)

In [ ]:
df_students['course_id'].unique()

In [ ]:
len(df_students['user_id'].unique())

In [ ]:
df_students.set_index(['course_id', 'section_id'], inplace=True)

In [ ]:
df_students.head()

In [ ]:
df_students.loc['TICT-V1PROG'].head()

In [ ]:
df_students.loc[ [('TICT-V1PROG', 'TICT-ICT-V1A'),('TICT-V1CSN', 'TICT-ICT-V1B')]]

In [ ]:
len(df_users)

In [ ]:
len(df_students)

## Mergen en groeperen

We gaan de gegevens van de Canvasgebruikers samenvoegen met de gegevens uit Osiris, zodat we weten welke Canvasgebruiker zich heeft ingeschreven voor welk vak en tot welke klas hij/zij behoort.

In [ ]:
df_students.reset_index(inplace=True)

In [ ]:
df_students.set_index(['course_id', 'user_id'], inplace=True)

In [ ]:
df_users.merge(df_students, how='inner', left_index=True, right_index=True).head()

In [ ]:
len(df_users.merge(df_students, how='inner', left_index=True, right_index=True))

Zonder gegevensverlies moeten we een _left join_ gebruiken.

In [ ]:
df_users.merge(df_students, how='left', left_index=True, right_index=True).head()

In [ ]:
len(df_users.merge(df_students, how='outer', left_index=True, right_index=True))

In [ ]:
df_all_users = df_users.merge(df_students, how='inner', left_index=True, right_index=True)

In [ ]:
df_all_users.head()

In [ ]:
df_prog.head()

In [ ]:
df_attendance = df_prog.merge(df_all_users, how='inner', 
                              left_on=['Student ID', 'Course Code'], 
                              right_on=['canvas_id','course_id'])

In [ ]:
df_attendance.head()

In [ ]:
df_attendance.size

In [ ]:
df_attendance['Attendance'].unique()

In [ ]:
df_attendance['Aanwezig'] = (df_attendance['Attendance'] != 'absent')

In [ ]:
df_attendance.head()

Laten we de aanwezigheid groeperen op vak, klas en datum.

In [ ]:
df_attendance_group = df_attendance.groupby(['Course Name', 'section_id', 'Class Date']).sum()

In [ ]:
df_attendance_group.head(15)

Gemiddelde aanwezigheid per datum (m.a.w. per les).

In [ ]:
df_attendance_group.groupby(['section_id']).mean()

## `datetime` data type

Laten we eens groeperen alléén op dag.

In [ ]:
df_attendance_daily = df_attendance.drop(['canvas_id', 'Student ID'], axis=1).groupby(['Class Date']).sum()

In [ ]:
df_attendance_daily.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_daily.index, df_attendance_daily['Aanwezig'])

In [ ]:
df_attendance_daily.index

De x-as (dag) is blijkbaar nominaal geïnterpreteerd ('object'), terwijl tijd natuurlijk gewoon kwantitatief is: laten we het datatype wijzigen om dat te reflecteren.

In [ ]:
df_attendance_daily.index = pd.to_datetime(df_attendance_daily.index, format='%d-%m-%Y')

In [ ]:
df_attendance_daily.index

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_daily.index, df_attendance_daily['Aanwezig'])

Het `datetime` type kunnen we nu ook gaan resamplen, bijvoorbeeld naar 'per week'.

In [ ]:
df_attendance_weekly = df_attendance_daily.resample('W', convention='start').sum()

In [ ]:
df_attendance_weekly.index

In [ ]:
df_attendance_weekly.index.week

In [ ]:
fig, ax = plt.subplots(figsize=(12,4), dpi=150)
ax.bar(df_attendance_weekly.index.week, df_attendance_weekly['Aanwezig'])